<a href="https://colab.research.google.com/github/ruperty/colab/blob/master/optimizer_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.python.framework import dtypes
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.util.tf_export import keras_export

In [2]:
@keras_export("keras.optimizers.Ecoli")
class Ecoli(optimizer_v2.OptimizerV2):
  """Ecoli optimizer.

  Computes:
  ```
  if loss has increased
  theta(t+1) = theta(t) - learning_rate * loss * random_gradient
  ```

  # References
      Powers, WT
      http://www.livingcontrolsystems.com/demos/ecoli/ecoli.pdf
  """

  def __init__(self,
               learning_rate=0.01,
               name="Ecoli",
               **kwargs):
    """Construct a new Ecoli optimizer.

    Arguments:
      learning_rate: float hyperparameter >= 0. Learning rate.
      name: Optional name prefix for the operations created when applying
        gradients.  Defaults to 'Ecoli'.
      **kwargs: keyword arguments. Allowed to be {`clipnorm`, `clipvalue`, `lr`,
        `decay`}. `clipnorm` is clip gradients by norm; `clipvalue` is clip
        gradients by value, `decay` is included for backward compatibility to
        allow time inverse decay of learning rate. `lr` is included for backward
        compatibility, recommended to use `learning_rate` instead.
    """
    super(Ecoli, self).__init__(name, **kwargs)
    self.loss_historical=tf.Variable(0.0)
    self.learning_rate=learning_rate

  def minimize(self, loss, var_list, grad_loss=None, name=None):
        grads_and_vars = self._compute_gradients(
            loss, var_list=var_list, grad_loss=grad_loss)
    
        return self.apply_gradients(grads_and_vars, name=name)

  def _resource_apply_dense(self, grad, var, apply_state=None):
      #print(grad.numpy())
      #self.minimize( loss, var):
      tf.print(grad)      
      tf.print(var)      
      return apply_ecoli(var, self.learning_rate, grad, self.loss_historical)

  def _compute_gradients(self, loss, var_list, grad_loss=None):
        loss_value = loss()
        print(loss_value)
        print(self.loss_historical)
        print(var_list)
        
        if loss_value >= self.loss_historical:
            grads = tf.random.uniform(shape=(len(var_list),), minval=-1., maxval=1.)
            
        self.loss_historical.assign(loss_value)
        
        print(grads)
        #var_list += grads
        grads_and_vars = list(zip(grads, var_list))
        self._assert_valid_dtypes([
            v for g, v in grads_and_vars
            if g is not None and v.dtype != dtypes.resource
        ])
    
        return grads_and_vars

  def _create_slots(self, var_list):
    pass #x=0 # do nothing



  def get_config(self):
    config = super(Ecoli, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
    })
    return config

In [6]:
# I am trying to apply the weight changes here, but not sure if that is what this function is for
def apply_ecoli(var, lr, grad, loss):
    
    tf.print("apply_ecoli")
    result = var + lr * grad * loss
    return result

In [5]:
class Linear(layers.Layer):
    def __init__(self,   weight_init=1, name='Linear'):
        super(Linear, self).__init__(name=name)
        self.wi=weight_init
        
    def build(self, input_shape):
        self.w = tf.Variable(
            initial_value=tf.constant(np.full((1,1), self.wi), dtype=tf.float32 ),
            trainable=True,
        )

    def call(self, inputs):
        self.add_metric(self.w[0], name='weight', aggregation='mean')
        return inputs * self.w[0] 
    
    def get_config(self):
        return {'wi': self.wi}


In [10]:
scale=1
reference=0
row = np.array([1, 2, 5])        
print(row)

[1 2 5]


In [11]:
X = tf.constant(  np.array([row]), dtype=tf.float32 )/scale
Y = tf.constant(np.full((1,row.size), reference), dtype=tf.float32 )/scale
dataset = tf.data.Dataset.from_tensor_slices(( X , Y )) 
dataset = dataset.shuffle( 1 ).repeat( 1 ).batch( 1 )
iterator = dataset.__iter__()
x_batch , y_batch = iterator.get_next()
print(X)

tf.Tensor([[1. 2. 5.]], shape=(1, 3), dtype=float32)


In [15]:
model = Sequential( [ Linear( 0.5 ) ])
model.build(x_batch.shape)

In [17]:
optimizer = Ecoli(0.001)
model.compile(loss='mse', optimizer=optimizer)


In [18]:
history = model.fit(  x_batch , y_batch , epochs=3, verbose=1)

Epoch 1/3
[[10.000001]]
[[0.5]]
apply_ecoli
1/1 [==============================] - 0s 2ms/step - loss: 2.5000 - weight: 0.5000
Epoch 2/3
[[10.000001]]
[[0.5]]
apply_ecoli
1/1 [==============================] - 0s 2ms/step - loss: 2.5000 - weight: 0.5000
Epoch 3/3
[[10.000001]]
[[0.5]]
apply_ecoli
1/1 [==============================] - 0s 3ms/step - loss: 2.5000 - weight: 0.5000
